## Preprocessing for School Location Database


In [80]:
import os
import pandas as pd
from shapely.geometry import Point
from pyspark.sql import SparkSession

# Create a spark session (which will run spark jobs)
spark = (
    SparkSession.builder.appName("MAST30034 Tutorial 1")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config("spark.driver.memory", "6g")
    .getOrCreate()
)

# Directory containing the school CSV files
schools_dir = '../data/landing/schools'

# List all CSV files in the directory
csv_files = [f for f in os.listdir(schools_dir) if f.endswith('.csv')]

# Load each CSV and collect their columns
schemas = {}
for file in csv_files:
    try:
        df = pd.read_csv(os.path.join(schools_dir, file), nrows=0, encoding='utf-8')
    except UnicodeDecodeError:
        df = pd.read_csv(os.path.join(schools_dir, file), nrows=0, encoding='latin1')
    schemas[file] = set(df.columns)

# Display the columns for each file
for file, cols in schemas.items():
    print(f"{file}: {sorted(cols)}")

# Check if all schemas match
all_schemas = list(schemas.values())
schemas_match = all(s == all_schemas[0] for s in all_schemas)
print(f"\nSchemas match: {schemas_match}")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/10/10 13:48:40 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/10/10 13:48:40 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


school_locations_2023.csv: ['Address_Line_1', 'Address_Line_2', 'Address_Postcode', 'Address_State', 'Address_Town', 'Education_Sector', 'Entity_Type', 'Full_Phone_No', 'LGA_ID', 'LGA_Name', 'Postal_Address_Line_1', 'Postal_Address_Line_2', 'Postal_Postcode', 'Postal_State', 'Postal_Town', 'School_Name', 'School_No', 'School_Status', 'School_Type', 'X', 'Y']
school_locations_2025.csv: ['Address_Line_1', 'Address_Line_2', 'Address_Postcode', 'Address_State', 'Address_Town', 'Area', 'Education_Sector', 'Entity_Type', 'Full_Phone_No', 'LGA_ID', 'LGA_Name', 'LGA_TYPE', 'Postal_Address_Line_1', 'Postal_Address_Line_2', 'Postal_Postcode', 'Postal_State', 'Postal_Town', 'Region', 'School_Name', 'School_No', 'School_Status', 'School_Type', 'X', 'Y']
school_locations_2024.csv: ['AREA_Name', 'Address_Line_1', 'Address_Line_2', 'Address_Postcode', 'Address_State', 'Address_Town', 'Education_Sector', 'Entity_Type', 'Full_Phone_No', 'LGA_ID', 'LGA_Name', 'Postal_Address_Line_1', 'Postal_Address_Lin

In [81]:
# make pandas show all rows when printing dataframe
pd.set_option('display.max_rows', None)

In [82]:
# Define the standardized schema based on common columns across all years
standard_columns = [
    'Address_Line_1', 'Address_Line_2', 'Address_Postcode', 'Address_State', 'Address_Town',
    'Education_Sector', 'Entity_Type', 'Full_Phone_No', 'LGA_ID', 'LGA_Name',
    'Postal_Address_Line_1', 'Postal_Address_Line_2', 'Postal_Postcode', 'Postal_State', 'Postal_Town',
    'School_Name', 'School_No', 'School_Type', 'X', 'Y',
    # Additional columns that exist in some years
    'Area', 'LGA_TYPE', 'Region', 'School_Status'
]

print("Standardized schema:")
for col in standard_columns:
    print(f"  - {col}")


Standardized schema:
  - Address_Line_1
  - Address_Line_2
  - Address_Postcode
  - Address_State
  - Address_Town
  - Education_Sector
  - Entity_Type
  - Full_Phone_No
  - LGA_ID
  - LGA_Name
  - Postal_Address_Line_1
  - Postal_Address_Line_2
  - Postal_Postcode
  - Postal_State
  - Postal_Town
  - School_Name
  - School_No
  - School_Type
  - X
  - Y
  - Area
  - LGA_TYPE
  - Region
  - School_Status


In [83]:
# Function to standardize column names and add missing columns
def standardize_school_dataframe(df, year):
    """
    Standardize a school dataframe to have consistent columns.
    
    Parameters:
    df: pandas DataFrame
    year: string indicating the year (2023, 2024, or 2025)
    
    Returns:
    pandas DataFrame with standardized columns
    """
    # Create a copy to avoid modifying the original
    df_std = df.copy()
    
    # Handle column name variations
    column_mapping = {
        'AREA_Name': 'Area',  # 2024 has AREA_Name instead of Area
        'Region_Name': 'Region'  # 2024 has Region_Name instead of Region
    }
    
    # Rename columns
    df_std = df_std.rename(columns=column_mapping)
    
    # Add missing columns with NaN values
    for col in standard_columns:
        if col not in df_std.columns:
            df_std[col] = None
    
    # Reorder columns to match standard schema
    df_std = df_std[standard_columns]
    
    # Add year column to indicate when school was established
    df_std['establishment_year'] = year
    
    return df_std

# Test the function with a small sample
print("Testing standardization function...")
for file in csv_files:
    year = file.split('_')[-1].split('.')[0]  # Extract year from filename
    print(f"\nProcessing {file} (year: {year})")
    
    # Load a small sample to test
    try:
        df_sample = pd.read_csv(os.path.join(schools_dir, file), nrows=5, encoding='utf-8')
    except UnicodeDecodeError:
        df_sample = pd.read_csv(os.path.join(schools_dir, file), nrows=5, encoding='latin1')
    
    # Standardize
    df_std = standardize_school_dataframe(df_sample, year)
    
    print(f"Original columns: {len(df_sample.columns)}")
    print(f"Standardized columns: {len(df_std.columns)}")
    print(f"Missing columns added: {[col for col in standard_columns if col not in df_sample.columns]}")


Testing standardization function...

Processing school_locations_2023.csv (year: 2023)
Original columns: 21
Standardized columns: 25
Missing columns added: ['Area', 'LGA_TYPE', 'Region']

Processing school_locations_2025.csv (year: 2025)
Original columns: 24
Standardized columns: 25
Missing columns added: []

Processing school_locations_2024.csv (year: 2024)
Original columns: 22
Standardized columns: 25
Missing columns added: ['Area', 'LGA_TYPE', 'Region', 'School_Status']


In [84]:
# Load and standardize all school datasets
print("Loading and standardizing all school datasets...")
standardized_dfs = []

for file in csv_files:
    year = file.split('_')[-1].split('.')[0]  # Extract year from filename
    print(f"\nProcessing {file} (year: {year})")
    
    # Load the full dataset
    try:
        df = pd.read_csv(os.path.join(schools_dir, file), encoding='utf-8')
    except UnicodeDecodeError:
        df = pd.read_csv(os.path.join(schools_dir, file), encoding='latin1')
    
    print(f"  Loaded {len(df)} schools")
    
    # Standardize the dataframe
    df_std = standardize_school_dataframe(df, year)
    standardized_dfs.append(df_std)
    
    print(f"  Standardized to {len(df_std.columns)} columns")

# Combine all standardized dataframes
print(f"\nCombining {len(standardized_dfs)} datasets...")
combined_schools = pd.concat(standardized_dfs, ignore_index=True)

print(f"Combined dataset shape: {combined_schools.shape}")
print(f"Total schools: {len(combined_schools)}")
print(f"\nEstablishment year distribution:")
print(combined_schools['establishment_year'].value_counts().sort_index())


Loading and standardizing all school datasets...

Processing school_locations_2023.csv (year: 2023)
  Loaded 2302 schools
  Standardized to 25 columns

Processing school_locations_2025.csv (year: 2025)
  Loaded 2301 schools
  Standardized to 25 columns

Processing school_locations_2024.csv (year: 2024)
  Loaded 2294 schools
  Standardized to 25 columns

Combining 3 datasets...
Combined dataset shape: (6897, 25)
Total schools: 6897

Establishment year distribution:
establishment_year
2023    2302
2024    2294
2025    2301
Name: count, dtype: int64


In [85]:
# select School_Name, School_No, School_Type, School_Status, establishment_year, X, Y
combined_schools = combined_schools[['School_Name', 'Education_Sector', 'School_Type', 'School_Status', 'establishment_year', 'X', 'Y']]

In [86]:
# convert the X, Y columns to a Point object from shapely 
combined_schools['coordinates'] = combined_schools.apply(lambda row: Point(row['X'], row['Y']), axis=1)

In [87]:
# round the X, Y columns to 4 decimal places
combined_schools['X'] = combined_schools['X'].round(1)
combined_schools['Y'] = combined_schools['Y'].round(1)


In [88]:
combined_schools['establishment_year'].value_counts()

establishment_year
2023    2302
2025    2301
2024    2294
Name: count, dtype: int64

In [89]:
combined_schools['School_Type'].value_counts()


School_Type
Primary      4715
Secondary    1072
Pri/Sec       746
Special       349
Language       13
Camp            2
Name: count, dtype: int64

In [90]:
# remove schools with the same fuzzy matched name and X and Y duplicates at 1 dp
from difflib import get_close_matches
to_drop = set()
name_to_indices = {}    
for idx, row in combined_schools.iterrows():
    name = row['School_Name']
    x = row['X']
    y = row['Y']

    # Find close matches to the current school name
    close_matches = get_close_matches(name, name_to_indices.keys(), n=1, cutoff=0.99)
    
    if close_matches:
        matched_name = close_matches[0]
        for matched_idx in name_to_indices[matched_name]:
            matched_row = combined_schools.loc[matched_idx]
            if matched_row['X'] == x and matched_row['Y'] == y:
                # If X and Y also match, mark the current index for dropping
                to_drop.add(idx)
                break
    
    # Add the current index to the mapping
    if name not in name_to_indices:
        name_to_indices[name] = []
    name_to_indices[name].append(idx)  
print(f"Dropping {len(to_drop)} duplicate schools based on fuzzy name and coordinates match.")
combined_schools = combined_schools.drop(index=to_drop).reset_index(drop=True)
print(f"Dataset shape after removing duplicates: {combined_schools.shape}")

Dropping 4534 duplicate schools based on fuzzy name and coordinates match.
Dataset shape after removing duplicates: (2363, 8)


In [91]:
# sort by school_name
combined_schools = combined_schools.sort_values(by='School_Name')

In [92]:
combined_schools['School_Type'].value_counts()

School_Type
Primary      1592
Secondary     373
Pri/Sec       258
Special       133
Language        5
Camp            2
Name: count, dtype: int64

In [93]:
# remove 'School_Type' where it is 'Language', 'Camp' 
combined_schools = combined_schools[~combined_schools['School_Type'].isin(['Language', 'Camp'])]

combined_schools['School_Type'].value_counts()

School_Type
Primary      1592
Secondary     373
Pri/Sec       258
Special       133
Name: count, dtype: int64

In [94]:
combined_schools['School_Status'].value_counts()

School_Status
O    2354
Name: count, dtype: int64

In [95]:
# remove 'School_Status' where it is 'Closed'
combined_schools = combined_schools[combined_schools['School_Status'] != 'C']

In [96]:
# drop School_Status column
combined_schools = combined_schools.drop(columns=['School_Status'])



In [97]:
# convert column names to lower casing
combined_schools.columns = combined_schools.columns.str.lower()

combined_schools.head(5)

,school_name,education_sector,school_type,establishment_year,x,y,coordinates
318,Abbotsford Primary School,Government,Primary,2023,145.0,-37.8,POINT (144.99881 -37.80898)
644,Aberfeldie Primary School,Government,Primary,2023,144.9,-37.8,POINT (144.89477 -37.75649)
1655,Academy of Mary Immaculate,Catholic,Secondary,2023,145.0,-37.8,POINT (144.97441 -37.80371)
1855,Adass Israel School,Independent,Pri/Sec,2023,145.0,-37.9,POINT (145.0082 -37.8839)
2257,Advance College of Education Incorporated,Independent,Special,2023,144.9,-38.4,POINT (144.88506 -38.38285)


In [98]:
# check for duplicate school_name
duplicate_schools = combined_schools[combined_schools.duplicated(subset=['school_name'])]

duplicate_schools


,school_name,education_sector,school_type,establishment_year,x,y,coordinates
1828,Corpus Christi School,Catholic,Primary,2023,144.9,-37.8,POINT (144.87292 -37.80853)
2110,Corpus Christi School,Catholic,Primary,2023,144.7,-37.9,POINT (144.66344 -37.90995)
2156,Emmaus Catholic Primary School,Catholic,Primary,2023,144.8,-37.7,POINT (144.77224 -37.7003)
1920,Holy Family School,Catholic,Primary,2023,145.2,-38.0,POINT (145.23863 -37.9905)
1870,Holy Family School,Catholic,Primary,2023,144.3,-38.1,POINT (144.33833 -38.1111)
1739,Holy Rosary School,Catholic,Primary,2023,144.3,-36.7,POINT (144.31057 -36.72487)
1734,Holy Rosary School,Catholic,Primary,2023,144.9,-37.8,POINT (144.92878 -37.79558)
2011,Holy Spirit School,Catholic,Primary,2023,145.2,-37.8,POINT (145.23692 -37.79323)
2175,Lumen Christi School,Catholic,Primary,2023,144.8,-37.9,POINT (144.75334 -37.8926)
2132,Lumen Christi School,Catholic,Primary,2023,143.8,-37.6,POINT (143.81724 -37.58019)


In [99]:
# group duplicate_schools by school_name, education_sector, school_type, establishment_year and count the number of occurences
duplicate_schools_grouped = duplicate_schools.groupby(['school_name', 'education_sector', 'school_type', 'establishment_year']).size().reset_index(name='count')

# get the school_name from duplicate_schools_grouped where count is greater than 1
duplicate_school_names = duplicate_schools_grouped[duplicate_schools_grouped['count'] > 1]['school_name']

duplicate_school_names

0                  Corpus Christi School
2                     Holy Family School
3                     Holy Rosary School
5                   Lumen Christi School
6                         Marian College
8     Our Lady Help of Christians School
12                  Sacred Heart College
13                   Sacred Heart School
16                      St Anne's School
17                   St Anthony's School
18                 St Augustine's School
20                   St Bernard's School
21                   St Brendan's School
22                    St Brigid's School
27                   St Dominic's School
28              St Francis Xavier School
32                      St James' School
34                      St John's School
36                    St Joseph's School
43                      St Mary's School
44                   St Michael's School
45                    St Monica's School
46                   St Patrick's School
47                      St Paul's School
48              

In [100]:
combined_schools_unique = combined_schools[~combined_schools['school_name'].isin(duplicate_school_names)]

combined_schools_unique[combined_schools_unique['school_name'].duplicated()]

,school_name,education_sector,school_type,establishment_year,x,y,coordinates
2156,Emmaus Catholic Primary School,Catholic,Primary,2023,144.8,-37.7,POINT (144.77224 -37.7003)
2011,Holy Spirit School,Catholic,Primary,2023,145.2,-37.8,POINT (145.23692 -37.79323)
2343,OneSchool Global Vic,Independent,Pri/Sec,2025,144.2,-36.8,POINT (144.228738 -36.781544)
1948,Our Lady of Lourdes School,Catholic,Primary,2023,145.3,-37.8,POINT (145.26576 -37.84898)
1909,Our Lady's School,Catholic,Primary,2023,146.3,-36.4,POINT (146.32114 -36.37219)
2022,Resurrection School,Catholic,Primary,2023,144.8,-37.7,POINT (144.77539 -37.7363)
1735,St Aloysius' School,Catholic,Primary,2023,144.7,-38.3,POINT (144.6553 -38.26868)
2059,St Andrew's School,Catholic,Primary,2023,145.1,-37.9,POINT (145.10719 -37.93489)
1845,St Bernadette's School,Catholic,Primary,2023,145.0,-37.8,POINT (145.03412 -37.76306)
1750,St Catherine's School,Independent,Pri/Sec,2023,145.0,-37.8,POINT (145.02127 -37.8383)


In [101]:
coburg_special_developmental_school = combined_schools_unique[combined_schools_unique['school_name']=="Coburg Special Developmental School"]


In [102]:
coburg_special_developmental_school

,school_name,education_sector,school_type,establishment_year,x,y,coordinates
1014,Coburg Special Developmental School,Government,Special,2023,145.0,-37.7,POINT (144.95174 -37.73292)


**Dont know why but doesnt run and i think some preprocess graph so not importnat**

In [103]:
import folium

# get the x, y coordinates of coburg_special_developmental_school
x1, y1 = coburg_special_developmental_school['x'].iloc[0], coburg_special_developmental_school['y'].iloc[0]
x2, y2 = coburg_special_developmental_school['x'].iloc[1], coburg_special_developmental_school['y'].iloc[1]


# plot the points (x1, y1) and (x2, y2) on a Folium map by converting to a Point object from shapely
from shapely.geometry import Point

point1 = Point(x1, y1)
point2 = Point(x2, y2)

# plot points on a Folium map
m = folium.Map(location=[y1, x1], zoom_start=15)
folium.Marker(location=[y1, x1], popup='Point 1').add_to(m)
folium.Marker(location=[y2, x2], popup='Point 2').add_to(m)
m


IndexError: single positional indexer is out-of-bounds

In [104]:
combined_schools_duplicates = combined_schools[combined_schools['school_name'].isin(duplicate_school_names)]

In [105]:
# Save the combined dataset
output_path = '../data/processed/schools/schools.csv'
os.makedirs(os.path.dirname(output_path), exist_ok=True)

combined_schools.to_csv(output_path, index=False)


**School Rank Code**

In [106]:
# Scrape this website for top 100 schools in victoria when school type is secondary and 101 for schools that are not in the top 100. If the school type
# is not secondry set the column to None
# https://bettereducation.com.au/Results/vce.aspx

import io
import pandas as pd
import requests

headers = {
    "User-Agent": (
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) "
        "AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.0 Safari/605.1.15"
    ),
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
    "Accept-Language": "en-AU,en;q=0.9",
    "Referer": "https://bettereducation.com.au/Results/vce.aspx",
    "Cache-Control": "no-cache",
}
url = "https://bettereducation.com.au/Results/vce.aspx"
resp = requests.get(url, headers=headers, timeout=30)

resp.raise_for_status()

tables = pd.read_html(io.StringIO(resp.text))  # no match -> gets all tables

In [107]:
school_rank_df = tables[1].copy()
# make the Better Education Rank columns go from 1 to 100 including every number in between
school_rank_df["Better Education Rank"] = range(1, 101)
school_rank_df

,Unnamed: 0,Better Education Rank,School,scores of 40+ (%),Median VCE Score,BE Percentile,Locality,IB,Unit 3-4 cohort,Trend / Compare
0,1,1,Ballarat Clarendon College,34.7,37,NaN,BALLARAT,NaN,255,Trend / Compare
1,2,2,Huntingtower School,31.7,37,NaN,MOUNT WAVERLEY,NaN,182,Trend / Compare
2,3,3,MacRobertson Girls High School,31.1,37,NaN,MELBOURNE,Find IB results,566,Trend / Compare
3,4,4,Mount Scopus Memorial College,30.8,37,NaN,BURWOOD,NaN,248,Trend / Compare
4,5,5,St Catherine's School,30.8,36,NaN,TOORAK,NaN,132,Trend / Compare
5,6,6,Ruyton Girls' School,30.2,37,NaN,KEW,NaN,154,Trend / Compare
6,7,7,Haileybury College (Girls),29.9,36,NaN,KEYSBOROUGH,NaN,434,Trend / Compare
7,8,8,Bialik College,29.7,36,NaN,HAWTHORN,NaN,171,Trend / Compare
8,9,9,Melbourne Girls Grammar,28.7,36,NaN,SOUTH YARRA,NaN,248,Trend / Compare
9,10,10,Melbourne High School,28.5,36,NaN,SOUTH YARRA,NaN,705,Trend / Compare


In [108]:
from difflib import get_close_matches
import re

# Match the schools.csv with the scraped data and add a new column called vic_secondary_rank
# If the school is in the top 100, set the rank to the rank from the scraped data
# If the school type is secondary or primary/secondary and the school is not in the top 100, set the rank to 101
# If the school type is not secondary or primary/secondary, set the rank to None
school_rank_df.columns = [c.lower().strip() for c in school_rank_df.columns]
school_rank_df = school_rank_df.rename(columns={"better education rank": "vic_secondary_rank", "school": "school_name"})
school_rank_df["school_name"] = school_rank_df["school_name"].str.strip()
school_rank_df = school_rank_df[pd.to_numeric(school_rank_df["vic_secondary_rank"], errors="coerce").notna()]
school_rank_df["vic_secondary_rank"] = school_rank_df["vic_secondary_rank"].astype(int) 

schools = pd.read_csv("../data/processed/schools/schools.csv")
def normalize(name: str) -> str:
    if not isinstance(name, str):
        return ""
    cleaned = (
        name.lower()
            .replace("’", "'")
            .replace("–", "-")
            .strip()
    )
    # drop everything after the first comma (suburb/campus info)
    cleaned = cleaned.split(",", 1)[0]
    cleaned = re.sub(r"\s+", " ", cleaned)
    return cleaned

# ranking table
school_rank_df["name_norm"] = school_rank_df["school_name"].map(normalize)
lookup_norm = dict(zip(school_rank_df["name_norm"], school_rank_df["vic_secondary_rank"]))

# schools.csv
schools["school_name_norm"] = schools["school_name"].map(normalize)

def lookup_fuzzy(name, candidates, cutoff=0.8):
    matches = get_close_matches(name, candidates, n=1, cutoff=cutoff)
    return matches[0] if matches else None

candidate_names = list(lookup_norm.keys())

def assign_rank(row):
    rank = lookup_norm.get(row["school_name_norm"])
    if rank is None:
        # try fuzzy match against known names (casefolded) and goes both way for better matching
        match = lookup_fuzzy(row["school_name_norm"], candidate_names, cutoff=0.85)
        if match:
            rank = lookup_norm[match]
    school_type = row["school_type"].strip().lower()
    if rank:
        return rank
    if school_type not in {"secondary", "pri/sec"}:
        return None
    return 101

schools["vic_secondary_rank"] = schools.apply(assign_rank, axis=1)
schools = schools.drop(columns=["school_name_norm"])
schools.to_csv("../data/processed/schools/schools_ranked.csv", index=False)

In [109]:
set(range(1, 101)) - set(schools["vic_secondary_rank"].unique())

{7, 30, 47, 51, 58, 85, 88, 91}

In [110]:
schools[schools["school_name"].str.contains("Haileybury College", case=False, na=False)]

,school_name,education_sector,school_type,establishment_year,x,y,coordinates,vic_secondary_rank
732,Haileybury College,Independent,Pri/Sec,2023,145.1,-38.0,POINT (145.14643 -37.99598),15.0


In [111]:
import numpy as np
eps = 0.0001
# make a school goodness column based on the vic_secondary_rank
def school_goodness(rank):
    if pd.isna(rank):
        return "N/A"
    else:
        goodness = 1 - (np.log(rank) / (np.log(101)) + eps) # Normalized log rank
        return round(goodness, 4)
    
schools["school_goodness"] = schools["vic_secondary_rank"].map(school_goodness)
schools.to_csv("../data/processed/schools/schools_ranked.csv", index=False)
schools

,school_name,education_sector,school_type,establishment_year,x,y,coordinates,vic_secondary_rank,school_goodness
0,Abbotsford Primary School,Government,Primary,2023,145.0,-37.8,POINT (144.99881 -37.80898),NaN,N/A
1,Aberfeldie Primary School,Government,Primary,2023,144.9,-37.8,POINT (144.89477 -37.75649),NaN,N/A
2,Academy of Mary Immaculate,Catholic,Secondary,2023,145.0,-37.8,POINT (144.97441 -37.80371),87.0,0.0322
3,Adass Israel School,Independent,Pri/Sec,2023,145.0,-37.9,POINT (145.0082 -37.8839),101.0,-0.0001
4,Advance College of Education Incorporated,Independent,Special,2023,144.9,-38.4,POINT (144.88506 -38.38285),NaN,N/A
5,Ainslie Parklands Primary School,Government,Primary,2023,145.3,-37.8,POINT (145.26424 -37.79379),NaN,N/A
6,Aintree Primary School,Government,Primary,2023,144.7,-37.7,POINT (144.66623 -37.7238),NaN,N/A
7,Aireys Inlet Primary School,Government,Primary,2023,144.1,-38.5,POINT (144.10997 -38.45737),NaN,N/A
8,Airly Primary School,Government,Primary,2023,147.1,-38.0,POINT (147.1139 -38.02571),NaN,N/A
9,Aitken College,Independent,Pri/Sec,2023,144.9,-37.6,POINT (144.8907 -37.62771),101.0,-0.0001


**Best School per Isochrone**

In [294]:
import pandas as pd
import geopandas as gpd
from pyproj import Geod
from shapely import wkt, ops

def safe_wkt(value):
    if pd.isna(value):
        return None
    value = str(value).strip()
    if not value:
        return None
    try:
        return wkt.loads(value)
    except Exception:
        return None

# ---- Load + parse ----
spark_df_loaded = spark.read.parquet("../data/curated/rent_features/cleaned_listings_sampled_parquet")
df_loaded = spark_df_loaded.toPandas()

# Convert WKT back to geometry
from shapely import wkt
df_loaded['coordinates'] = df_loaded['coordinates_wkt'].apply(wkt.loads)
df_loaded = df_loaded.drop(columns=['coordinates_wkt'])

# Convert back to GeoDataFrame
listings_gdf = gpd.GeoDataFrame(df_loaded, geometry='coordinates')


iso_columns = [c for c in listings_gdf.columns if c.endswith("min_imputed") or c.endswith("min")]

def to_geom(val):
    if val is None or val in ("nan", "None"):
        return None
    # handle both strings and already-parsed geometries
    return wkt.loads(val) if isinstance(val, str) else val

for col in iso_columns:
    listings_gdf[f"geom_{col}"] = listings_gdf[col].apply(to_geom)


listings_gdf["year"] = (
    listings_gdf["year"]
    .astype("Int64")
)
def swap_axes(geom):
    return ops.transform(lambda x, y, z=None: (y, x), geom)


listings_gdf["listing_point"] = (
    listings_gdf["coordinates"]
        .apply(safe_wkt)                                  # raw WKT -> Point/None
        .apply(lambda g: swap_axes(g) if g is not None else None)
)


schools["establishment_year"] = pd.to_numeric(schools["establishment_year"], errors="coerce").apply(lambda g: int(g))
schools["geometry"] = schools["coordinates"].apply(safe_wkt)
schools["coordinates"] = schools["coordinates"].apply(safe_wkt)
schools_gdf = gpd.GeoDataFrame(schools, geometry="geometry", crs="EPSG:4326")

geod = Geod(ellps="WGS84")
beta = 0.2 # equivalent to lambda = 1/ beta 
def score_row(goodness, dist_km):
    return goodness / (1 + beta * dist_km)

In [295]:
iso_columns

['driving_5min',
 'driving_10min',
 'driving_15min',
 'walking_5min',
 'walking_10min',
 'walking_15min',
 'driving_5min_imputed',
 'driving_10min_imputed',
 'driving_15min_imputed',
 'walking_5min_imputed',
 'walking_10min_imputed',
 'walking_15min_imputed']

In [296]:
pairs = {}
for col in iso_columns:
    if col.endswith("_imputed"):
        base = col.replace("_imputed", "")
        if base in listings_gdf.columns:
            pairs[base] = (base, col)

missing_pairs = pd.Series(False, index=listings_gdf.index)

for base, (orig, imp) in pairs.items():
    both_null = listings_gdf[[orig, imp]].isna().all(axis=1)
    if both_null.any():
        print(f"{base}: {both_null.sum()} rows missing both original and imputed")
        missing_pairs |= both_null

rows_missing_any_pair = listings_gdf.loc[missing_pairs]
print("Rows missing every column in at least one pair:", len(rows_missing_any_pair))

# Adding a flag feature for listings with both original and imputed missing

pairs = [
    (base, f"{base}_imputed")
    for base in iso_columns
    if base in listings_gdf.columns and f"{base}_imputed" in listings_gdf.columns
]

missing_iso = pd.Series(False, index=listings_gdf.index)
for base, imp in pairs:
    missing_iso |= listings_gdf[[base, imp]].isna().all(axis=1)

listings_gdf["missing_iso_feature"] = missing_iso.astype("int8")

driving_5min: 644 rows missing both original and imputed
driving_10min: 644 rows missing both original and imputed
driving_15min: 644 rows missing both original and imputed
walking_5min: 1553 rows missing both original and imputed
walking_10min: 1553 rows missing both original and imputed
walking_15min: 1553 rows missing both original and imputed
Rows missing every column in at least one pair: 2141


In [297]:
best_cols = [c for c in listings_gdf.columns if c.startswith("geom_") and "_imputed" not in c]
iso_columns_imp = [c for c in iso_columns if "_imputed" not in c]
# imputes the isochrone and the geom column with nearest neighbour if they are empty
for col in best_cols:
    imp = f"{col}_imputed"
    if imp in listings_gdf.columns:
        listings_gdf[col] = listings_gdf[col].fillna(listings_gdf[imp]).infer_objects(copy=False)

for col in iso_columns_imp:
    imp = f"{col}_imputed"
    if imp in listings_gdf.columns:
        listings_gdf[col] = listings_gdf[col].fillna(listings_gdf[imp]).infer_objects(copy=False)


to_drop1 = [c for c in listings_gdf.columns if c.endswith("min_imputed")]


In [298]:
listings_gdf = listings_gdf.drop(columns=to_drop1)

In [299]:
listings_gdf.head()

,property_id,rental_price,bedrooms,bathrooms,car_spaces,property_type,suburb,year,quarter,longitude,...,walking_15min,coordinates,geom_driving_5min,geom_driving_10min,geom_driving_15min,geom_walking_5min,geom_walking_10min,geom_walking_15min,listing_point,missing_iso_feature
0,17742020,860.0,2,2,1,apartment / unit / flat,melbourne,2025,3,144.959739,...,"POLYGON ((144.947523 -37.810827, 144.947463 -3...",POINT (-37.809 144.96),"POLYGON ((144.940233 -37.803985, 144.940299 -3...","POLYGON ((144.906098 -37.806776, 144.907918 -3...","POLYGON ((144.861498 -37.826183, 144.862484 -3...","POLYGON ((144.955663 -37.810727, 144.955681 -3...","POLYGON ((144.951502 -37.810486, 144.951813 -3...","POLYGON ((144.947523 -37.810827, 144.947463 -3...",POINT (144.95974 -37.80931),0
1,17654223,570.0,3,2,3,House,narre warren-hampton park,2025,2,145.294162,...,"POLYGON ((145.282052 -38.01112, 145.283925 -38...",POINT (-38.016 145.294),"POLYGON ((145.257832 -38.006234, 145.262623 -3...","POLYGON ((145.222915 -37.962763, 145.224211 -3...","POLYGON ((145.157882 -37.909223, 145.159778 -3...","POLYGON ((145.291435 -38.017998, 145.291573 -3...","POLYGON ((145.287877 -38.018162, 145.287952 -3...","POLYGON ((145.282052 -38.01112, 145.283925 -38...",POINT (145.29416 -38.01624),0
2,17416488,1680.0,5,3,1,House,brighton,2025,1,144.999444,...,"POLYGON ((144.987827 -37.909434, 144.988023 -3...",POINT (-37.911 144.999),"POLYGON ((144.983543 -37.902391, 144.983274 -3...","POLYGON ((144.976182 -37.882672, 144.979855 -3...","POLYGON ((144.970316 -37.864392, 144.97033 -37...","POLYGON ((144.995763 -37.913244, 144.995807 -3...","POLYGON ((144.992308 -37.912345, 144.992259 -3...","POLYGON ((144.987827 -37.909434, 144.988023 -3...",POINT (144.99944 -37.91108),0
3,17718606,800.0,3,2,2,apartment / unit / flat,altona,2025,3,144.806045,...,"POLYGON ((144.794638 -37.867949, 144.794809 -3...",POINT (-37.87 144.806),"POLYGON ((144.800033 -37.859679, 144.800425 -3...","POLYGON ((144.769485 -37.841431, 144.769204 -3...","POLYGON ((144.701669 -37.886116, 144.707818 -3...","POLYGON ((144.80198 -37.871874, 144.802231 -37...","POLYGON ((144.798753 -37.87392, 144.799112 -37...","POLYGON ((144.794638 -37.867949, 144.794809 -3...",POINT (144.80604 -37.86959),0
4,16837560,1200.0,3,2,1,apartment / unit / flat,melbourne,2025,3,144.954228,...,"POLYGON ((144.943373 -37.813404, 144.943084 -3...",POINT (-37.815 144.954),"POLYGON ((144.934094 -37.809439, 144.934092 -3...","POLYGON ((144.901143 -37.804204, 144.902488 -3...","POLYGON ((144.83131 -37.827485, 144.835401 -37...","POLYGON ((144.951876 -37.81517, 144.9519 -37.8...","POLYGON ((144.946636 -37.814711, 144.946638 -3...","POLYGON ((144.943373 -37.813404, 144.943084 -3...",POINT (144.95423 -37.81489),0


In [300]:
subset = listings_gdf.iloc[:, -15:]      # grab the trailing 30 columns
mask = subset.isna().all(axis=1)         # True where all 30 are null/NaN
count_empty = mask.sum()                 # how many such rows

empty_rows = listings_gdf.loc[mask]      # inspect the actual rows
print("Rows with all of the last 30 columns empty:", count_empty)
empty_rows.head()                                # or empty_rows[subset.columns] to focus on that slice

Rows with all of the last 30 columns empty: 0


,property_id,rental_price,bedrooms,bathrooms,car_spaces,property_type,suburb,year,quarter,longitude,...,walking_15min,coordinates,geom_driving_5min,geom_driving_10min,geom_driving_15min,geom_walking_5min,geom_walking_10min,geom_walking_15min,listing_point,missing_iso_feature


In [301]:
iso_columns2 = [c for c in listings_gdf.columns if c.endswith("min") and "geom_" not in c]

iso_columns2

['driving_5min',
 'driving_10min',
 'driving_15min',
 'walking_5min',
 'walking_10min',
 'walking_15min']

In [302]:
all_results = []
iso_columns2 = [c for c in listings_gdf.columns if c.endswith("min") and "geom_" not in c]

for col in iso_columns2:
    poly_col = f"geom_{col}"
    # activate polygon geometry and drop rows without polygons
    iso_poly = listings_gdf.set_geometry(poly_col)
    iso_poly = iso_poly[iso_poly[poly_col].notna()]
    iso_poly = iso_poly.set_crs("EPSG:4326", allow_override=True)  # define if missing
    # (optionally ensure validity)
    # iso_poly[poly_col] = iso_poly[poly_col].buffer(0)

    # spatial join: schools inside polygon (keep listing_point column intact)
    joined = gpd.sjoin(
        iso_poly,
        schools_gdf,
        how="left",
        predicate="covers",
        rsuffix="school",
    )

    # only keep schools that existed by first_listed_year (or unknown year)
    mask = joined["establishment_year"].isna() | (
        joined["establishment_year"] <= joined["year"]
    )
    joined = joined[mask]

    if not joined.empty:
        # geodesic distance from listing point to school (FIX 3 & 4)
        lon1 = joined["listing_point"].apply(lambda g: g.x).values
        lat1 = joined["listing_point"].apply(lambda g: g.y).values
        lon2 = gpd.GeoSeries(joined["coordinates_school"]).x
        lat2 = gpd.GeoSeries(joined["coordinates_school"]).y
        _, _, dists_m = geod.inv(lon1, lat1, lon2, lat2)
        joined["dist_km"] = dists_m / 1000.0
    
        joined["school_goodness"] = pd.to_numeric(joined["school_goodness"], errors="coerce")
        joined["dist_km"] = pd.to_numeric(joined["dist_km"], errors="coerce")

        valid = joined["school_goodness"].notna() & joined["dist_km"].notna()
        joined["score"] = pd.NA
        joined.loc[valid, "score"] = score_row(joined.loc[valid, "school_goodness"], joined.loc[valid, "dist_km"])

        # a count of how many schools within a given isochrone
        count = (joined.groupby("property_id")["school_name"]
                .count()                                 # counts rows in each group
                .rename(f"n_schools_{col}")             # e.g., n_schools_driving_5min
                .to_frame())

        best_inside = (
            joined.dropna(subset=["score"])
            .sort_values("score", ascending=False)
            .groupby("property_id")
            .head(1)
            .set_index("property_id")
        )

        # build out indexed by property_id
        out = (iso_poly[["property_id"]]
            .drop_duplicates()
            .set_index("property_id"))
        for c in [f"best_school_name_{col}", f"best_school_coord_{col}", f"best_score_{col}", f"best_dist_km_{col}"]:
            out[c] = None

        if not best_inside.empty:
            idx = best_inside.index
            out.loc[idx, f"best_school_name_{col}"]  = best_inside["school_name"].to_numpy()
            out.loc[idx, f"best_school_coord_{col}"] = best_inside["coordinates_school"].to_numpy()
            out.loc[idx, f"best_score_{col}"]        = best_inside["score"].to_numpy()
            out.loc[idx, f"best_dist_km_{col}"]      = best_inside["dist_km"].to_numpy()



        # join back to listings_gdf by property_id
        all_results.append(out)
        all_results.append(count)

final_out = pd.concat(all_results, axis=1)  # columns are already unique per {col}
overlap = final_out.columns.intersection(listings_gdf.columns)
listings_gdf = (
    listings_gdf
    .drop(columns=overlap, errors="ignore")
    .join(final_out, on="property_id")
)


In [303]:
listings_gdf.head()

,property_id,rental_price,bedrooms,bathrooms,car_spaces,property_type,suburb,year,quarter,longitude,...,best_school_name_walking_10min,best_school_coord_walking_10min,best_score_walking_10min,best_dist_km_walking_10min,n_schools_walking_10min,best_school_name_walking_15min,best_school_coord_walking_15min,best_score_walking_15min,best_dist_km_walking_15min,n_schools_walking_15min
0,17742020,860.0,2,2,1,apartment / unit / flat,melbourne,2025,3,144.959739,...,Ozford College,POINT (144.95442 -37.81289),-0.000089,0.614315,2.0,Simonds Catholic College,POINT (144.95288 -37.80597),-0.000088,0.708616,7.0
1,17654223,570.0,3,2,3,House,narre warren-hampton park,2025,2,145.294162,...,None,None,None,None,0.0,None,None,None,None,2.0
2,17416488,1680.0,5,3,1,House,brighton,2025,1,144.999444,...,Brighton Primary School,POINT (145.00108 -37.91221),0.32771,0.190572,3.0,Brighton Primary School,POINT (145.00108 -37.91221),0.32771,0.190572,3.0
3,17718606,800.0,3,2,2,apartment / unit / flat,altona,2025,3,144.806045,...,None,None,None,None,0.0,Mount St Joseph Girls' College,POINT (144.81167 -37.8639),-0.000086,0.802222,1.0
4,16837560,1200.0,3,2,1,apartment / unit / flat,melbourne,2025,3,144.954228,...,Ozford College,POINT (144.95442 -37.81289),-0.000096,0.22245,2.0,Simonds Catholic College,POINT (144.95288 -37.80597),-0.000083,0.996974,4.0


**Filling in Walking and Driving best columns with nearest neighbour column logic**

In [304]:
listings_gdf.columns

Index(['property_id', 'rental_price', 'bedrooms', 'bathrooms', 'car_spaces',
       'property_type', 'suburb', 'year', 'quarter', 'longitude',
       ...
       'best_school_name_walking_10min', 'best_school_coord_walking_10min',
       'best_score_walking_10min', 'best_dist_km_walking_10min',
       'n_schools_walking_10min', 'best_school_name_walking_15min',
       'best_school_coord_walking_15min', 'best_score_walking_15min',
       'best_dist_km_walking_15min', 'n_schools_walking_15min'],
      dtype='object', length=321)

In [305]:
fields = ["best_school_name", "best_school_coord", "best_score", "best_dist_km"]

def all_fields_present(df, token):
    mode, dur = token.split("_", 1)
    cols = [f"{f}_{mode}_{dur}" for f in fields if f"{f}_{mode}_{dur}" in df.columns]
    if not cols:
        return pd.Series(False, index=df.index)
    return df[cols].notna().all(axis=1)

# availability for every token we might touch
tokens = [
    "walking_5min", "walking_10min", "walking_15min",
    "driving_5min", "driving_10min", "driving_15min",
]
avail = {tok: all_fields_present(listings_gdf, tok) for tok in tokens}

# per-target fallback order
fallback = {
    "walking_5min":  ["walking_10min", "walking_15min", "driving_5min", "driving_10min", "driving_15min"],
    "walking_10min": ["walking_15min", "driving_5min", "driving_10min", "driving_15min"],
    "walking_15min": ["driving_5min", "driving_10min", "driving_15min"],
    "driving_5min":  ["driving_10min", "driving_15min"],
    "driving_10min": ["driving_5min", "driving_15min"],
    "driving_15min": ["driving_10min", "driving_5min"],
}

for target in ["walking_5min", "walking_10min", "driving_5min", "driving_10min", "driving_15min"]:
    mode_tgt, dur_tgt = target.split("_", 1)
    target_cols = [f"{f}_{mode_tgt}_{dur_tgt}" for f in fields if f"{f}_{mode_tgt}_{dur_tgt}" in listings_gdf.columns]
    if not target_cols:
        continue

    missing = listings_gdf[target_cols].isna().all(axis=1)
    filled = pd.Series(False, index=listings_gdf.index)

    for candidate in fallback.get(target, []):
        rows = missing & avail[candidate] & ~filled
        if not rows.any():
            continue

        mode_src, dur_src = candidate.split("_", 1)
        for f in fields:
            src_col = f"{f}_{mode_src}_{dur_src}"
            tgt_col = f"{f}_{mode_tgt}_{dur_tgt}"
            if src_col in listings_gdf.columns and tgt_col in listings_gdf.columns:
                listings_gdf.loc[rows, tgt_col] = listings_gdf.loc[rows, src_col].values

        tgt_cnt = f"n_schools_{mode_tgt}_{dur_tgt}"
        src_cnt = f"n_schools_{mode_src}_{dur_src}"
        if tgt_cnt in listings_gdf.columns and src_cnt in listings_gdf.columns:
            need = rows & listings_gdf[tgt_cnt].fillna(0).eq(0)
            listings_gdf.loc[need, tgt_cnt] = listings_gdf.loc[need, src_cnt].values

        filled |= rows  # stop once we copy from the first available candidate



In [306]:
score_cols = [c for c in listings_gdf.columns if c.startswith("best_score_")]
listings_gdf[score_cols] = listings_gdf[score_cols].fillna(3e-8) # Justification can be asked by venura(formula with reasonably max args)
dist_cols = [c for c in listings_gdf.columns if c.startswith("best_dist_km_")]
for col in dist_cols:
    max_val = listings_gdf[col].max(skipna=True)
    listings_gdf[col] = listings_gdf[col].fillna(max_val + 1 if pd.notna(max_val) else 1.0)

/var/folders/qf/25308l_17hv3chqx70wpnbqm0000gn/T/ipykernel_5962/1073041575.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  listings_gdf[score_cols] = listings_gdf[score_cols].fillna(3e-8) # Justification can be asked by venura(formula with reasonably max args)
/var/folders/qf/25308l_17hv3chqx70wpnbqm0000gn/T/ipykernel_5962/1073041575.py:6: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  listings_gdf[col] = listings_gdf[col].fillna(max_val + 1 if pd.notna(max_val) else 1.0)


In [307]:
mask = listings_gdf.iloc[:, -18:].isna().any(axis=1)

# should be zero as all missing values are imputed
len(listings_gdf.loc[mask])

8964

In [308]:
# convert back to a csv file
listings_gdf.to_csv("../data/curated/rent_features/cleaned_listings_isochrones_added_with_best_schools.csv", index=False)

In [310]:
listings_gdf.head()

,property_id,rental_price,bedrooms,bathrooms,car_spaces,property_type,suburb,year,quarter,longitude,...,best_school_name_walking_10min,best_school_coord_walking_10min,best_score_walking_10min,best_dist_km_walking_10min,n_schools_walking_10min,best_school_name_walking_15min,best_school_coord_walking_15min,best_score_walking_15min,best_dist_km_walking_15min,n_schools_walking_15min
0,17742020,860.0,2,2,1,apartment / unit / flat,melbourne,2025,3,144.959739,...,Ozford College,POINT (144.95442 -37.81289),-0.000089,0.614315,2.0,Simonds Catholic College,POINT (144.95288 -37.80597),-8.758691e-05,0.708616,7.0
1,17654223,570.0,3,2,3,House,narre warren-hampton park,2025,2,145.294162,...,Fountain Gate Secondary College,POINT (145.29538 -38.00528),-0.000080,1.221659,7.0,None,None,3.000000e-08,3.693937,2.0
2,17416488,1680.0,5,3,1,House,brighton,2025,1,144.999444,...,Brighton Primary School,POINT (145.00108 -37.91221),0.327710,0.190572,3.0,Brighton Primary School,POINT (145.00108 -37.91221),3.277096e-01,0.190572,3.0
3,17718606,800.0,3,2,2,apartment / unit / flat,altona,2025,3,144.806045,...,Mount St Joseph Girls' College,POINT (144.81167 -37.8639),-0.000086,0.802222,1.0,Mount St Joseph Girls' College,POINT (144.81167 -37.8639),-8.617389e-05,0.802222,1.0
4,16837560,1200.0,3,2,1,apartment / unit / flat,melbourne,2025,3,144.954228,...,Ozford College,POINT (144.95442 -37.81289),-0.000096,0.222450,2.0,Simonds Catholic College,POINT (144.95288 -37.80597),-8.337538e-05,0.996974,4.0


In [309]:
output_path = "../data/curated/rent_features/cleaned_listings_with_schools.parquet"
geom_cols = [c for c in listings_gdf.columns if c.startswith("geom_")]

for col in geom_cols:
    listings_gdf[col] = listings_gdf[col].apply(lambda g: g.wkt if g is not None else None)

listings_gdf.to_parquet(output_path, index=False)


ArrowInvalid: ('Could not convert <POINT (144.946 -37.793)> with type Point: did not recognize Python value type when inferring an Arrow data type', 'Conversion failed for column best_school_coord_driving_5min with type object')